In [1]:
# dependencies to run the notebook

# !pip install torch==1.12.1
# !pip install torchmetrics==0.10.2
# !pip install torchvision==0.14.0
# !pip install texttable==1.6.4

<span style="color:darkviolet">
<font size="3">Download the below files from https://drive.google.com/drive/folders/1q50QMurzK9a5l4JBHWjf8VuWcZkbF7PM to run this notebook : <br>
1) train_bert_embeddings.pkl <br>
2) test_bert_embeddings.pkl <br> </font>
</span>

In [53]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from torchmetrics import Accuracy, F1Score, Precision, Recall
from texttable import Texttable
from torch.utils.data.sampler import SubsetRandomSampler
warnings.filterwarnings('ignore')
import os

In [54]:
# os.getcwd()

In [4]:
# Read train BERT embeddings
with open('../../../../../train_roberta_embeddings.pkl', 'rb') as f:
    training_data = pickle.load(f)

In [5]:
# Read test BERT embeddings
with open('../../../../../test_roberta_embeddings.pkl', 'rb') as f:
    testing_data = pickle.load(f)

In [6]:
# check
for item in training_data:
    print('The data is : ', item)

The data is :  embeddings
The data is :  tokenized_txt


In [7]:
# check
training_data['tokenized_txt'][0]

['Ġcontent',
 'Ġlicense',
 'Ġand',
 'Ġintellectual',
 'Ġproperty',
 'Ġrights',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 

In [8]:
len(training_data['embeddings']), training_data['embeddings'][0].shape

(7531, torch.Size([1, 512, 768]))

In [9]:
len(training_data['tokenized_txt']) , len(training_data['tokenized_txt'][0])

(7531, 512)

<span style="color:darkviolet">
<font size="4">Get the labels from train and test files.</font>
</span>

In [10]:
train_dataset_path = "../legal_bert/data/tos_clauses_train.csv"
test_dataset_path = "../legal_bert/data/tos_clauses_dev.csv"

In [11]:
train_df = pd.read_csv(train_dataset_path, header=0)
test_df = pd.read_csv(test_dataset_path, header=0)

In [12]:
train_targets = train_df.label.values
test_targets = test_df.label.values

In [13]:
test_targets

array([0, 0, 0, ..., 0, 1, 0])

In [14]:
# First checking if GPU is available
train_on_gpu=torch.backends.mps.is_available()
if(train_on_gpu):
    print('Training on GPU.')
    device = torch.device("mps")
else:
    print('No GPU available, training on CPU.')

Training on GPU.


<span style="color:darkviolet">
<font size="4">Create Dataset, Train and Test Classes</font>
</span>

In [15]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [16]:
class Train_Model(Dataset):
    
    def __init__(self, X, Y, transform=None):
        self.data1 = X
        self.data2 = Y
        self.transform = transform
        
    def __len__(self):
        return len(self.data1)
    
    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]
        
        
        if self.transform is not None:
            x = torch.tensor(x)
            
        return x, torch.tensor(y)                   

In [17]:
class Test_Model(Dataset):
    
    def __init__(self, X, transform=None):
        self.data1 = X
        self.transform = transform
        
    def __len__(self):
        return len(self.data1)
    
    def __getitem__(self, index):
        x = self.data1[index]
                
        if self.transform is not None:
            x = torch.tensor(x)
            
        return x

In [18]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = Train_Model(train_df['sentences'], train_df['label'])
# X_test_tensor = Train_Model(test_df)

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
# split = int(np.floor(num_train))
# train_idx = indices[split:]

train_sampler = SubsetRandomSampler(indices)
# valid_sampler = SubsetRandomSampler(valid_idx)
train_df_by_index = train_df.loc[indices]
# val_df_by_index = df_train.loc[valid_idx]
train_fair = sum(train_df_by_index['label'] == 0)
train_unfair = sum(train_df_by_index['label'] == 1)
# val_fair = sum(val_df_by_index['label'] == 0)
# val_unfair = sum(val_df_by_index['label'] == 1)

print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))
# print("val_fair:" + str(val_fair))
# print("val_unfair:" + str(val_unfair))

train_fair:6705
train_unfair:826


In [19]:
train_data = Train_Model(training_data['embeddings'],
                              train_targets,
                              transform=transforms.ToTensor())
test_data = Test_Model(testing_data['embeddings'],
                            transform=transforms.ToTensor())

<span style="color:darkviolet">
<font size="4">Prepare Data loaders</font>
</span>

In [20]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=batch_size,
                                             sampler=train_sampler,
                                             num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data,
                                            batch_size=batch_size,
                                            num_workers=num_workers)

In [21]:
# check sizes
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([20, 1, 512, 768])
Sample input: 
 tensor([[[[ 0.0066,  0.0683, -0.0492,  ..., -0.1372, -0.0292,  0.0263],
          [-0.2287,  0.0436,  0.2377,  ...,  0.3928,  0.0439, -0.1978],
          [-0.0615,  0.0450,  0.0209,  ...,  0.1682,  0.1016,  0.1232],
          ...,
          [ 0.1794,  0.1406, -0.0955,  ...,  0.0755, -0.0396, -0.0895],
          [ 0.0464,  0.1520, -0.2502,  ...,  0.0440, -0.1282,  0.0411],
          [ 0.0203,  0.0635, -0.1043,  ..., -0.2142, -0.0215,  0.0154]]],


        [[[-0.0157,  0.0656, -0.0627,  ..., -0.1670, -0.0277,  0.0606],
          [ 0.1689,  0.2124, -0.0917,  ...,  0.2360, -0.1986,  0.0415],
          [ 0.0319,  0.1823, -0.1980,  ...,  0.5952,  0.2550,  0.0426],
          ...,
          [ 0.1710,  0.1172, -0.0953,  ..., -0.1375, -0.0492, -0.0842],
          [-0.0206,  0.0414, -0.2403,  ...,  0.1024, -0.1325, -0.0109],
          [-0.0103,  0.0523, -0.1137,  ..., -0.2497, -0.0230,  0.0599]]],


        [[[ 0.0226,  0.0603, -0.

<span style="color:darkviolet">
<font size="5">SIMPLE RNN</font><br>
<font size="2.5">Number of hidden dimension : 20</font> <br>
<font size="2.5">Number of layers: 1</font> <br>
<font size="2.5">Number of epochs: 5</font> <br>
</span>

In [22]:
class RNNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(RNNet, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=1, batch_first=True, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

In [23]:
EMBEDDING_DIM = 768
HIDDEN_DIM = 20
OUTPUT_DIM = 2

model_rnn = RNNet(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_rnn)

RNNet(
  (rnn): RNN(768, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [24]:
def train_RNN(model, train_loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in train_loader:
        
        train  = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()
                
        output = model(train)
        
        loss = criterion(output, labels) 
        
        acc = binary_accuracy(output.argmax(-1), labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [25]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    correct = (preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [27]:
N_EPOCHS = 5

best_valid_loss = float('inf')

criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1/train_fair, 1/train_unfair]))
# criterion = nn.NLLLoss()

optimizer_rnn = optim.Adam(model_rnn.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    rnn_train_loss, rnn_train_acc = train_RNN(model_rnn,
                                      train_loader,
                                      optimizer_rnn,
                                      criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: \
    {rnn_train_loss:.3f} | Train Acc: {rnn_train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 22s
	Train Loss:     0.686 | Train Acc: 88.89%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss:     0.677 | Train Acc: 88.53%
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss:     0.669 | Train Acc: 86.25%
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss:     0.656 | Train Acc: 79.78%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss:     0.646 | Train Acc: 74.41%


In [28]:
test_loader_predict = torch.utils.data.DataLoader(
    test_data,
    batch_size=1,
    num_workers=0)

def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test  = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions_rnn = predict(model_rnn, test_loader_predict)

a_tensor = torch.IntTensor(predictions_rnn)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
rnn_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average='macro') # checked if weighted can be used
rnn_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average='macro', num_classes=2)
rnn_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average='macro', num_classes=2)
rnn_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of simple RNN :", rnn_test_acc)
print("F1 score of simple RNN :", rnn_f1_score)
print("Precision of simple RNN :", rnn_precision)
print("Recall of simple RNN :", rnn_recall)

Accuracy of simple RNN : 0.7419012188911438
F1 score of simple RNN : 0.5697938203811646
Precision of simple RNN : 0.56910240650177
Recall of simple RNN : 0.635807991027832


In [29]:
table = Texttable()
table.set_cols_dtype(['a','f','f','f','f','f'])
table.set_precision(5)
table.add_rows([["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
                ["Simple RNN", rnn_train_acc, rnn_test_acc, rnn_f1_score, rnn_precision, rnn_recall]])
print(table.draw(),"\n")

+------------+----------------+---------------+----------+-----------+---------+
|   Model    | Train accuracy | Test Accuracy | F1-score | Precision | Recall  |
+============+================+===============+==========+===========+=========+
| Simple RNN | 0.74408        | 0.74190       | 0.56979  | 0.56910   | 0.63581 |
+------------+----------------+---------------+----------+-----------+---------+ 



<span style="color:darkviolet">
<font size="5">Gated RNN</font><br>
</span>

In [30]:
class GRU_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(GRU_Network, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=1, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

In [31]:
model_gru = GRU_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_gru)

GRU_Network(
  (rnn): GRU(768, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [32]:
def train_GRU(model, train_loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in train_loader:
        
        train  = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()
                
        output = model(train)
        
        loss = criterion(output, labels) 
        
        acc = binary_accuracy(output.argmax(-1), labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [33]:
N_EPOCHS = 5

best_valid_loss = float('inf')

criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1/train_fair, 1/train_unfair]))
# criterion = nn.NLLLoss()

optimizer_gru = optim.Adam(model_gru.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    gru_train_loss, gru_train_acc = train_RNN(model_gru,
                                      train_loader,
                                      optimizer_gru,
                                      criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: \
    {gru_train_loss:.3f} | Train Acc: {gru_train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 30s
	Train Loss:     0.689 | Train Acc: 80.77%
Epoch: 02 | Epoch Time: 0m 25s
	Train Loss:     0.672 | Train Acc: 80.43%
Epoch: 03 | Epoch Time: 0m 26s
	Train Loss:     0.660 | Train Acc: 75.23%
Epoch: 04 | Epoch Time: 0m 27s
	Train Loss:     0.646 | Train Acc: 74.34%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss:     0.635 | Train Acc: 73.11%


In [34]:
test_loader_predict = torch.utils.data.DataLoader(
    test_data,
    batch_size=1,
    num_workers=0)

def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test  = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions_gru = predict(model_gru, test_loader_predict)

a_tensor = torch.IntTensor(predictions_gru)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
gru_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average='macro') # checked if weighted can be used
gru_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average='macro', num_classes=2)
gru_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average='macro', num_classes=2)
gru_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of Gated RNN :", gru_test_acc)
print("F1 score of Gated RNN :", gru_f1_score)
print("Precision of Gated RNN :", gru_precision)
print("Recall of Gated RNN :", gru_recall)

Accuracy of Gated RNN : 0.7546468377113342
F1 score of Gated RNN : 0.5704818964004517
Precision of Gated RNN : 0.5669399499893188
Recall of Gated RNN : 0.6238023042678833


In [35]:
table = Texttable()
table.set_cols_dtype(['a','f','f','f','f','f'])
table.set_precision(5)
table.add_rows([["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
                ["Gated RNN", gru_train_acc, gru_test_acc, gru_f1_score, gru_precision, gru_recall]])
print(table.draw(),"\n")

+-----------+----------------+---------------+----------+-----------+---------+
|   Model   | Train accuracy | Test Accuracy | F1-score | Precision | Recall  |
+===========+================+===============+==========+===========+=========+
| Gated RNN | 0.73113        | 0.75465       | 0.57048  | 0.56694   | 0.62380 |
+-----------+----------------+---------------+----------+-----------+---------+ 



<span style="color:darkviolet">
<font size="5">LSTM</font><br>
</span>

In [36]:
# LSTM

class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(LSTM_Network, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim))
        
        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [37]:
def train_LSTM(model, train_loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in train_loader:
        
        train  = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()
                
        output = model(train)
        
        loss = criterion(output, labels) 
        
        acc = binary_accuracy(output.argmax(-1), labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [38]:
model_lstm = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_lstm)

LSTM_Network(
  (lstm): LSTM(768, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=2, bias=True)
)


In [39]:
N_EPOCHS = 5

best_valid_loss = float('inf')

criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1/train_fair, 1/train_unfair]))

optimizer = optim.Adam(model_lstm.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    lstm_train_loss, lstm_train_acc = train_LSTM(model_lstm,
                                      train_loader,
                                      optimizer,
                                      criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: \
    {lstm_train_loss:.3f} | Train Acc: {lstm_train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 31s
	Train Loss:     0.687 | Train Acc: 71.66%
Epoch: 02 | Epoch Time: 0m 29s
	Train Loss:     0.667 | Train Acc: 77.96%
Epoch: 03 | Epoch Time: 0m 28s
	Train Loss:     0.659 | Train Acc: 72.32%
Epoch: 04 | Epoch Time: 0m 27s
	Train Loss:     0.649 | Train Acc: 70.26%
Epoch: 05 | Epoch Time: 0m 28s
	Train Loss:     0.639 | Train Acc: 68.76%


In [40]:
test_loader_predict = torch.utils.data.DataLoader(
    test_data,
    batch_size=1,
    num_workers=num_workers)

def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test  = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions_lstm = predict(model_lstm, test_loader_predict)

a_tensor = torch.IntTensor(predictions_lstm)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
lstm_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average='macro') # checked if weighted can be used
lstm_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average='macro', num_classes=2)
lstm_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average='macro', num_classes=2)
lstm_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of LSTM :", lstm_test_acc)
print("F1 score of LSTM :", lstm_f1_score)
print("Precision of LSTM :", lstm_precision)
print("Recall of of LSTM :", lstm_recall)

Accuracy of LSTM : 0.6909187436103821
F1 score of LSTM : 0.5431151390075684
Precision of LSTM : 0.5592541098594666
Recall of of LSTM : 0.6327338218688965


In [41]:
table = Texttable()
table.set_cols_dtype(['a','f','f','f','f','f'])
table.set_precision(5)
table.add_rows([["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
                ["LSTM", lstm_train_acc, lstm_test_acc, lstm_f1_score, lstm_precision, lstm_recall]])
print(table.draw(),"\n")

+-------+----------------+---------------+----------+-----------+---------+
| Model | Train accuracy | Test Accuracy | F1-score | Precision | Recall  |
+=======+================+===============+==========+===========+=========+
| LSTM  | 0.68761        | 0.69092       | 0.54312  | 0.55925   | 0.63273 |
+-------+----------------+---------------+----------+-----------+---------+ 



<span style="color:darkviolet">
<font size="5">Bi-LSTM</font><br>
</span>

In [42]:
# LSTM

class Bi_LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(Bi_LSTM_Network, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))
        c0 = Variable(torch.zeros(1 * 2, x.size(0), self.hidden_dim))
        
        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [43]:
def train_Bi_LSTM(model, train_loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in train_loader:
        
        train  = Variable(batch[0].view(-1, 512, 768))
        labels = Variable(batch[1])

        optimizer.zero_grad()
                
        output = model(train)
        
        loss = criterion(output, labels) 
        
        acc = binary_accuracy(output.argmax(-1), labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(train_loader), epoch_acc / len(train_loader)

In [44]:
model_bi_lstm = Bi_LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(model_bi_lstm)

Bi_LSTM_Network(
  (lstm): LSTM(768, 20, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=40, out_features=2, bias=True)
)


In [45]:
N_EPOCHS = 5

best_valid_loss = float('inf')

criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1/train_fair, 1/train_unfair]))

optimizer = optim.Adam(model_bi_lstm.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    bi_lstm_train_loss, bi_lstm_train_acc = train_Bi_LSTM(model_bi_lstm,
                                      train_loader,
                                      optimizer,
                                      criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: \
    {bi_lstm_train_loss:.3f} | Train Acc: {bi_lstm_train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 50s
	Train Loss:     0.687 | Train Acc: 83.90%
Epoch: 02 | Epoch Time: 0m 47s
	Train Loss:     0.673 | Train Acc: 81.49%
Epoch: 03 | Epoch Time: 0m 49s
	Train Loss:     0.652 | Train Acc: 72.41%
Epoch: 04 | Epoch Time: 0m 48s
	Train Loss:     0.643 | Train Acc: 77.69%
Epoch: 05 | Epoch Time: 0m 47s
	Train Loss:     0.633 | Train Acc: 74.51%


In [46]:
test_loader_predict = torch.utils.data.DataLoader(
    test_data,
    batch_size=1,
    num_workers=num_workers)

def predict(model, dataloader):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        test  = Variable(batch.view(-1, 512, 768))
        outputs = model(test)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
    return prediction_list

predictions_bi_lstm = predict(model_bi_lstm, test_loader_predict)

a_tensor = torch.IntTensor(predictions_bi_lstm)
b_tensor = torch.IntTensor(test_targets)

accuracy = Accuracy()
bi_lstm_test_acc = accuracy(a_tensor, b_tensor).item()

f1 = F1Score(num_classes=2, average='macro') # checked if weighted can be used
bi_lstm_f1_score = f1(a_tensor, b_tensor).item()

precision = Precision(average='macro', num_classes=2)
bi_lstm_precision = precision(a_tensor, b_tensor).item()

recall = Recall(average='macro', num_classes=2)
bi_lstm_recall = recall(a_tensor, b_tensor).item()

print("Accuracy of LSTM :", bi_lstm_test_acc)
print("F1 score of LSTM :", bi_lstm_f1_score)
print("Precision of LSTM :", bi_lstm_precision)
print("Recall of of LSTM :", bi_lstm_recall)

Accuracy of LSTM : 0.6123207807540894
F1 score of LSTM : 0.509839653968811
Precision of LSTM : 0.561344563961029
Recall of of LSTM : 0.6546074151992798


In [47]:
table = Texttable()
table.set_cols_dtype(['a','f','f','f','f','f'])
table.set_precision(5)
table.add_rows([["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
                ["Bi-LSTM", bi_lstm_train_acc, bi_lstm_test_acc, bi_lstm_f1_score, bi_lstm_precision, bi_lstm_recall]])
print(table.draw(),"\n")


+---------+----------------+---------------+----------+-----------+---------+
|  Model  | Train accuracy | Test Accuracy | F1-score | Precision | Recall  |
+=========+================+===============+==========+===========+=========+
| Bi-LSTM | 0.74506        | 0.61232       | 0.50984  | 0.56134   | 0.65461 |
+---------+----------------+---------------+----------+-----------+---------+ 



In [48]:
# compare all
table = Texttable()
table.set_cols_dtype(['a','f','f','f','f','f'])
table.set_precision(5)
table.add_rows([["Model", "Train accuracy", "Test Accuracy", "F1-score", "Precision", "Recall"],
                ["Simple RNN", rnn_train_acc, rnn_test_acc, rnn_f1_score, rnn_precision, rnn_recall],
                ["Gated RNN", gru_train_acc, gru_test_acc, gru_f1_score, gru_precision, gru_recall],
                ["LSTM", lstm_train_acc, lstm_test_acc, lstm_f1_score, lstm_precision, lstm_recall],
                ["Bi-LSTM", bi_lstm_train_acc, bi_lstm_test_acc, bi_lstm_f1_score, bi_lstm_precision, bi_lstm_recall]])
print(table.draw(),"\n")

+------------+----------------+---------------+----------+-----------+---------+
|   Model    | Train accuracy | Test Accuracy | F1-score | Precision | Recall  |
+============+================+===============+==========+===========+=========+
| Simple RNN | 0.74408        | 0.74190       | 0.56979  | 0.56910   | 0.63581 |
+------------+----------------+---------------+----------+-----------+---------+
| Gated RNN  | 0.73113        | 0.75465       | 0.57048  | 0.56694   | 0.62380 |
+------------+----------------+---------------+----------+-----------+---------+
| LSTM       | 0.68761        | 0.69092       | 0.54312  | 0.55925   | 0.63273 |
+------------+----------------+---------------+----------+-----------+---------+
| Bi-LSTM    | 0.74506        | 0.61232       | 0.50984  | 0.56134   | 0.65461 |
+------------+----------------+---------------+----------+-----------+---------+ 



<span style="color:darkviolet">
<font size="3">Best Performance : Gated-RNN</font><br>
</span>